# Hash table

Реализуйте собственную хеш-таблицу — структуру данных, которая хранит пары «ключ–значение» и позволяет выполнять три операции:
 • вставка элемента,
 • поиск по ключу,
 • удаление элемента.

Важно! Задание полностью творческое, чем больше аспектов будет учтено в реализации, тем лучше. 

На что стоит обратить внимание:
 • Как хранить данные? 
 • Как разрешать коллизии? 
 • Что делать, если таблица заполнится? 

Важно! Для хранения значений можно использовать только list!


# Import

In [1]:
import os

while os.getcwd().split("/")[-1] != "algorithms_python":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Implementation

In [9]:
class HashTable:
    """Реализация хеш-таблицы с линейным пробированием для разрешения коллизий."""

    def __init__(self, initial_capacity: int = 8, load_factor: float = 0.75):
        self.capacity = initial_capacity
        self.load_factor = load_factor
        self.size = 0
        self.keys = [None] * self.capacity
        self.values = [None] * self.capacity
        self._deleted = [False] * self.capacity  # Флаги для удаленных элементов

    def _calc_index(self, key) -> int:
        return hash(key) % self.capacity

    def _probe(self, key, find_empty: bool = False):
        """
        Линейное пробирование для поиска места.

        Args:
            key: ключ для поиска
            find_empty: искать ли пустую ячейку (для вставки)
        """
        index = self._calc_index(key)
        first_deleted = None

        for i in range(self.capacity):
            current_index = (index + i) % self.capacity

            if self.keys[current_index] is None:
                if find_empty:
                    return first_deleted if first_deleted is not None else current_index
                return None

            if self._deleted[current_index]:
                if first_deleted is None:
                    first_deleted = current_index
                continue

            if self.keys[current_index] == key:
                return current_index

        return None if not find_empty else first_deleted

    def _resize(self, new_capacity: int):
        old_keys = self.keys
        old_values = self.values
        old_deleted = self._deleted

        self.capacity = new_capacity
        self.keys = [None] * self.capacity
        self.values = [None] * self.capacity
        self._deleted = [False] * self.capacity
        self.size = 0

        for i in range(len(old_keys)):
            if old_keys[i] is not None and not old_deleted[i]:
                self.insert(old_keys[i], old_values[i])

    def insert(self, key, value) -> None:
        if self.size / self.capacity > self.load_factor:
            self._resize(self.capacity * 2)

        index = self._probe(key)

        if index is not None:
            # Ключ существует, обновляем значение
            self.values[index] = value
            if self._deleted[index]:
                self._deleted[index] = False
                self.size += 1
        else:
            # Новый ключ
            index = self._probe(key, find_empty=True)
            self.keys[index] = key
            self.values[index] = value
            self._deleted[index] = False
            self.size += 1

    def get(self, key, default=None):
        index = self._probe(key)
        if index is not None and not self._deleted[index]:
            return self.values[index]
        return default

    def delete(self, key) -> bool:
        index = self._probe(key)
        if index is not None and not self._deleted[index]:
            self._deleted[index] = True
            self.size -= 1

            if self.size / self.capacity < self.load_factor / 4 and self.capacity > 8:
                self._resize(max(8, self.capacity // 2))

            return True
        return False

    def __str__(self) -> str:
        """Отрисовка hash таблицы"""
        items = []
        for i in range(self.capacity):
            if self.keys[i] is not None and not self._deleted[i]:
                key_str = (
                    f"'{self.keys[i]}'"
                    if isinstance(self.keys[i], str)
                    else str(self.keys[i])
                )
                value_str = (
                    f"'{self.values[i]}'"
                    if isinstance(self.values[i], str)
                    else str(self.values[i])
                )
                items.append(f"{key_str}: {value_str}")

        return "{" + ", ".join(items) + "}"


table = HashTable()
table.insert("apple", 1)
table.insert("banana", 2)
table.insert("orange", 3)
print(table)

{'banana': 2, 'apple': 1, 'orange': 3}
